In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2024-09-09 07:47:11--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-09 07:47:11 (14.2 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-09-06 06:20:23--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-09-06 06:20:23 (65.2 MB/s) - ‘documents.json’ saved [658332/658332]



In [9]:
import minsearch

In [10]:
import json

In [11]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [12]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [13]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [54]:
index = minsearch.Index(
    text_fields = ["question","text","section"],
    keyword_fields = ["course"]
)

In [55]:
index.fit(documents)

In [82]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [84]:
import os
from dotenv import load_dotenv

# Reload the .env file
load_dotenv()

# Verify the new API key
api_key = os.getenv("OPENAI_API_KEY")


In [86]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [87]:
from openai import OpenAI

In [88]:
#get openai client
oppenai_client = OpenAI(api_key = api_key)

In [89]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}
    
    results = index.search(
        query = query,
        filter_dict = {'course' : 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )
    return results
    

In [90]:
def build_prompt(query, search_results):
    propmt_template = """
    You are a course teaching assistant. Anser the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the context when answering the questions. 
    
    QUESTION : {question}
    
    CONTEXT : {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section : {doc['section']}\nquestion: {doc['question']}\nanswer : {doc['text']}\n\n"

    prompt = propmt_template.format(question = query, context = context).strip()

    return prompt
        

In [91]:
def llm_response(prompt):
    response = oppenai_client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role":"user", "content": prompt}]
)
    return response.choices[0].message.content

In [92]:
def rag(query):
    results = search(query)
    prompt = build_prompt(query, results)
    response = llm_response(prompt)
    return response

In [106]:
query = "The course has already started. can I still enroll?"

In [107]:
answer = rag(query)

In [108]:
print(answer)

Yes, you can still enroll and join the course even after it has started. You're also eligible to submit the homework assignments. However, please be mindful of the deadlines for the final projects to avoid last-minute submissions.


In [1]:
pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [25]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch(
    'http://localhost:9200'
)

In [26]:
es_client.info()

ObjectApiResponse({'name': '39971c2048d9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'SSG_918tQ5idHjfGEIxuIQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [27]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"

In [ ]:
#creating index
es_client.indices.create(index=index_name,body=index_settings)

In [15]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [16]:
from tqdm.auto import tqdm

In [19]:
# indexing documents
for doc in documents:
    es_client.index(index=index_name, document=doc)
    

In [93]:
query = "How can I run Kafka?"

In [94]:
def elastic_search(query):
    search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
    search_result = es_client.search(index = index_name, body = search_query)
    
    result_docs = []
    for doc in search_result['hits']['hits']:
        result_docs.append(doc['_source'])
    return result_docs

In [95]:
def rag(query):
    results = elastic_search(query)
    prompt = build_prompt(query, results)
    response = llm_response(prompt)
    return response

In [96]:
response = rag(query)

In [97]:
response

'To run Kafka, you can follow the instructions provided for the specific task you want to achieve:\n\n1. **Running Java Kafka producer/consumer/kstreams/etc. in terminal:**\n   - Navigate to your project directory and use the following command:\n     ```shell\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n2. **Running a Python Kafka producer:**\n   - If you encounter the "Module \'kafka\' not found" error, you will need to create and use a virtual environment. Follow these steps:\n     - Create a virtual environment and install the required packages by running the following commands:\n       ```shell\n       python -m venv env\n       source env/bin/activate\n       pip install -r ../requirements.txt\n       ```\n     - Activate the virtual environment each time you need to run the Python file:\n       ```shell\n       source env/bin/activate\n       ```\n     - To deactivate the virtual environment:\n       ```shell\n